In [1]:
import re
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.linalg import norm
import jieba
from sklearn.cluster import KMeans
import pandas as pd
#特征提取
from sklearn.feature_extraction.text import TfidfTransformer
#特征提取
from sklearn.feature_extraction.text import CountVectorizer
#保存模型
import joblib
import random
from sklearn.cluster import DBSCAN
import csv
import numpy as np
import time
from collections import Counter

/Users/felixtan/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# 创建停用词list
def readwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

In [3]:
#生成中国城市区域名称
def citynamesgenerator():
    stopwords = readwordslist('./city.txt')  # 这里加载停用词的路径
    file_write_obj = open("citynames.txt", 'w',encoding='utf-8')
    for word in stopwords:
        cityname=word.split(' ')[1]
        # 以写的方式打开文件，如果文件不存在，就会自动创建
        file_write_obj.writelines(cityname)
        file_write_obj.write('\n')
        # 清洗城市级别前缀
        if cityname.endswith('特别行政区'): sub_cityname=cityname.rstrip('特别行政区')
        elif cityname.endswith('市辖区'): sub_cityname=cityname.rstrip('市辖区')
        elif cityname.endswith('省'): sub_cityname=cityname.rstrip('省')
        elif cityname.endswith('市'): sub_cityname=cityname.rstrip('市')
        else: sub_cityname=cityname
        #.rstrip('区').rstrip('县').rstrip('自治州')
        if sub_cityname!=cityname and sub_cityname!='':
            file_write_obj.writelines(sub_cityname)
            file_write_obj.write('\n')
    file_write_obj.close()

In [132]:
# 生成城市名称停用词
citynamesgenerator()

In [4]:
#调用停用词库，简化字符串，处理同义词
def simple(datas):
    result=[]
    # 添加公司名称停用词
    stopwords = readwordslist('./company_stopwords.txt') 
    # 地域名称通用词
    citynames=readwordslist('./citynames.txt')
    # 自定义通用词库
    permission_words=readwordslist('./addwords.txt')
    for data in datas:
        # 公司属性 公司性质停用词
        for stopword in stopwords:
            # 查找不到返回-1 查找到返回
            if data.find(stopword)!=-1:
                data=re.split(stopword,data)[0]
        # 过滤自定义词库
        flag=False
        for per_word in permission_words:
            if data.startswith(per_word):
                flag=True
        if flag==False:
            for cityname in citynames:
                # 地名前缀
                if data.startswith(cityname):
                    data=data.replace(cityname,'')
                    break
        for cityname in citynames:
            # 中插地名 e.g. （北京）
            test_chi='（ '+cityname+' ）'
            data=data.replace(test_chi,'')
            test_eng='( '+cityname+' )'
            data=data.replace(test_eng,'')
        result.append(data)
    return result

In [13]:
#生成特征矩阵
def get_TFIDF(corpus):
    # 将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在i类文本下的词频
    vectorizer = CountVectorizer()
    # 该类会统计每个词语的tf-idf权值
    transformer = TfidfTransformer()
    # 第一个fit_transform是计算tf-idf
    frequency=vectorizer.fit_transform(corpus)
    #第二个fit_transform是将文本转为词频矩阵
    tfidf = transformer.fit_transform(frequency)
    # 获取词袋模型中的所有词语
    word = vectorizer.get_feature_names()
    # 将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重
    weight = tfidf.toarray()
    return weight

In [14]:
#读取样本 提取公司名称
def excel_one_line_to_list():
    with open('./data10w.csv','r',encoding='gbk') as csvfile:
        reader = csv.reader(csvfile)
        df = [row[32] for row in reader]
    # 清理空字符串
    df = [string for string in df if string != ""]
    return df[1:10000]

In [16]:
#切割文本 过滤停用词 
def split_words(words):
    corpus = []  # 语料库 空格连接
    # 加载自定义词库
    permission_words=readwordslist("./addwords.txt")
    for per_word in permission_words:
        jieba.add_word(per_word)
    for word in words:
        # jieba分词
        text_cut=jieba.cut(word,cut_all=False)
        corpus.append(' '.join(text_cut))  
    return corpus

In [17]:
eps=[0.0000000000001]
# 读取公司名样本
data=excel_one_line_to_list()
#分词
corpus=split_words(data)
#停词
data_simplified=simple(corpus)
# print(data_simplified)
# data_simplified=[d.replace(' ','') for d in data_simplified]
print("文件读取完成，有效数据为： ",len(data))
for ep in eps:
    print("！————————当前尝试簇内距离为：",ep)
    #计时开始
    start=time.time()
    weight=get_TFIDF(data_simplified)
    print("特征提取结束")
    #eps为距离阈值ϵ，min_samples为邻域样本数阈值MinPts,X为数据
    outcome = DBSCAN(eps=ep, min_samples = 1).fit(weight)
    #结果提取
    labels=outcome.labels_
    print("聚类结束")
    #生成可视化表格
    count_cluster=Counter(outcome.labels_)
    count_company=Counter(data)

文件读取完成，有效数据为：  9999
！————————当前尝试簇内距离为： 1e-13
特征提取结束
聚类结束


In [147]:
# print(data_simplified)
label=[]
frequency=[]
companynames=[]
sim_company_names=[]
for l,f in count_cluster.items():
    #统计簇内元素的下标
    index=np.where(labels==l)[0]
    #统计公司名称
    test1=list(set([data[i] for i in index]))
    #统计简化公司名称
    test2=list(set([data_simplified[i] for i in index]))
    for c in test1:
        companynames.append(c)
        sim_company_names.append(test2)
        frequency.append(count_company[c])

st = pd.DataFrame()
st=pd.DataFrame()#(columns=('聚类名称','公司名称','公司简称','出现频数'))
st['公司名称']=companynames
st['简化公司名称']=sim_company_names
st['频数']=frequency
st.to_csv('./dataset/statistics-'+str(ep)+'-0201.csv',encoding='utf_8_sig',index=False)
# 生成公司聚类标签数据
df = pd.DataFrame()
df['公司名称']=data
df['简化公司名称']=data_simplified
df['标签'] = labels
df.to_csv('./dataset/cluster-'+str(ep)+'-0201.csv',encoding='utf_8_sig',index=False)

In [127]:

    label=[]
    frequency=[]
    companynames=[]
    sim_company_names=[]
    for l,f in count_cluster.items():
        #统计簇内元素的下标
        index=np.where(labels==l)[0]
        #统计公司名称
        test1=list(set([data[i] for i in index]))
        #统计简化公司名称
        test2=list(set([data_simplified[i] for i in index]))
        for c in test1:
            companynames.append(c)
            sim_company_names.append(test2)
            frequency.append(count_company[c])

    st = pd.DataFrame()
    st=pd.DataFrame()#(columns=('聚类名称','公司名称','公司简称','出现频数'))
    st['公司名称']=companynames
    st['简化公司名称']=sim_company_names
    st['频数']=frequency
    st.to_csv('./dataset/statistics-'+str(ep)+'.csv',encoding='utf_8_sig',index=False)
    # 生成公司聚类标签数据
    df = pd.DataFrame()
    df['公司名称']=data
    df['简化公司名称']=data_simplified
    df['标签'] = labels
    df.to_csv('./dataset/cluster-'+str(ep)+'.csv',encoding='utf_8_sig',index=False)
    print('文本生成结束')
    print("！————————总结")
    print("有效经纪公司总数：",len(data))
    print('聚类后有效公司数： ',len(companynames))
    print("关键词总数: ",weight.shape)
    total=(time.time())-start
    print("总耗时： ",total,"秒")
    print('共耗时： ',total/3600,'小时')

文件读取完成，有效数据为：  75353
！————————当前尝试簇内距离为： 0.001
！————————总结
有效经纪公司总数： 75353
聚类后有效公司数：  1569
关键词总数:  (75353, 1049)
总耗时：  10429.783033847809 秒
共耗时：  2.8971619538466133 小时
！————————当前尝试簇内距离为： 0.01
！————————总结
有效经纪公司总数： 75353
聚类后有效公司数：  1569
关键词总数:  (75353, 1049)
总耗时：  10563.932601928711 秒
共耗时：  2.9344257227579753 小时
！————————当前尝试簇内距离为： 0.1
！————————总结
有效经纪公司总数： 75353
聚类后有效公司数：  1569
关键词总数:  (75353, 1049)
总耗时：  10444.105413198471 秒
共耗时：  2.901140392555131 小时
！————————当前尝试簇内距离为： 1
！————————总结
有效经纪公司总数： 75353
聚类后有效公司数：  1569
关键词总数:  (75353, 1049)
总耗时：  17891.7833340168 秒
共耗时：  4.9699398150046665 小时


In [ ]:
# eps=[0.001,0.01,0.1,1]
# 读取公司名样本
data=excel_one_line_to_list()
print("文件读取完成，有效数据为： ",len(data))
print("！————————当前尝试簇内距离为：",ep)
#计时开始
start=time.time()
#停词
data_simplified=simple(data)
#     print("停词结束")
# 特征提取 计算TF-IDF
weight=get_TFIDF(data_simplified)
#     print("特征提取结束")
#eps为距离阈值ϵ，min_samples为邻域样本数阈值MinPts,X为数据
outcome = DBSCAN(eps=ep, min_samples = 1).fit(weight)
#结果提取
labels=outcome.labels_
#     print("聚类结束")
#生成可视化表格
count_cluster=Counter(outcome.labels_)
count_company=Counter(data)
label=[]
frequency=[]
companynames=[]
sim_company_names=[]
for l,f in count_cluster.items():
    #统计簇内元素的下标
    index=np.where(labels==l)[0]
    #统计公司名称
    test1=list(set([data[i] for i in index]))
    #统计简化公司名称
    test2=list(set([data_simplified[i] for i in index]))
    for c in test1:
        companynames.append(c)
        sim_company_names.append(test2)
        frequency.append(count_company[c])

st = pd.DataFrame()
st=pd.DataFrame()#(columns=('聚类名称','公司名称','公司简称','出现频数'))
st['公司名称']=companynames
st['简化公司名称']=sim_company_names
st['频数']=frequency
st.to_csv('./dataset/statistics-'+str(ep)+'.csv',encoding='utf_8_sig',index=False)
# 生成公司聚类标签数据
df = pd.DataFrame()
df['公司名称']=data
df['简化公司名称']=data_simplified
df['标签'] = labels
df.to_csv('./dataset/cluster-'+str(ep)+'.csv',encoding='utf_8_sig',index=False)
#     print('文本生成结束')
print("！————————总结")
print("有效经纪公司总数：",len(data))
print('聚类后有效公司数： ',len(companynames))
print("关键词总数: ",weight.shape)
total=(time.time())-start
print("总耗时： ",total,"秒")
print('共耗时： ',total/3600,'小时')